In [87]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
import scipy.io.wavfile as wavutils



In [97]:

"""Utils function related to signals for S3"""

def sigin(wavname:str)->[int,np.array]:
    """Functions that reads wave file and return sample rate and singal as np.array"""
    return wavutils.read(wavname,mmap=False)

def sawtooth(fs: int,Ns:int,Ss:int)->np.array:
    """Returns a Sawtooth wave of Sample rate Ss with Ns number of samples and Sample Frequency Fs"""
    Ss=np.linspace(0,1,Ss)
    return sp.signal.sawtooth(2 * np.pi * fs * Ss)[0:Ns]

def triangle(fs:int,Ns:int,Ss:int)->np.array:
    """Returns a Triangle wave of Sample rate Ss with Ns number of samples and Sample Frequency Fs"""
    Ss=np.linspace(0,1,Ss)
    return sp.signal.sawtooth(2 * np.pi * fs * Ss,0.5)[0:Ns]

def sin(fs:int,Ns:int,Ss:int)->np.array:
    """Returns a Sine wave of Sample rate Ss with Ns number of samples and Sample Frequency Fs"""
    t=np.arange(Ns)
    omega=2*np.pi*fs/Ss
    return np.sin(omega*t)

def cos(fs:int,Ns:int,Ss:int)->np.array:
    """Returns a Cosine wave of Sample rate Ss with Ns number of samples and Sample Frequency Fs"""
    t=np.arange(Ns)
    omega=2*np.pi*fs/Ss
    return np.cos(omega*t)


In [98]:
def create_FunctionFrame(fs:int,Ns:int,Ss:int)->pd.DataFrame:
    """Takes Sampling Frequency and returns a Data frame with function vectors of frequencies"""
    FuncFrame=pd.DataFrame()
    # Made a dictionary with key as func name and value ad function object
    Functions={"sin":sin,"cos":cos,"triangle":triangle,"sawtooth":sawtooth}

    #Iterated on dictionary items
    for i in Functions.items():
        for j in range(10):
            FuncFrame[i[0]+str(j)]=i[1](j*fs,Ns,Ss)

    return FuncFrame


In [99]:
create_FunctionFrame(1008,100,44100).head()

,sin0,sin1,sin2,sin3,sin4,sin5,sin6,sin7,sin8,sin9,...,sawtooth0,sawtooth1,sawtooth2,sawtooth3,sawtooth4,sawtooth5,sawtooth6,sawtooth7,sawtooth8,sawtooth9
0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-1.0,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
1,0.0,0.143122,0.283298,0.417641,0.543384,0.657939,0.758947,0.844328,0.912324,0.961536,...,-1.0,-0.954285,-0.908569,-0.862854,-0.817139,-0.771423,-0.725708,-0.679993,-0.634277,-0.588562
2,0.0,0.283298,0.543384,0.758947,0.912324,0.990950,0.988380,0.904827,0.747136,0.528227,...,-1.0,-0.908569,-0.817139,-0.725708,-0.634277,-0.542847,-0.451416,-0.359985,-0.268555,-0.177124
3,0.0,0.417641,0.758947,0.961536,0.988380,0.834573,0.528227,0.125333,-0.300468,-0.671351,...,-1.0,-0.862854,-0.725708,-0.588562,-0.451416,-0.314270,-0.177124,-0.039978,0.097168,0.234314
4,0.0,0.543384,0.912324,0.988380,0.747136,0.266037,-0.300468,-0.770513,-0.993200,-0.897038,...,-1.0,-0.817139,-0.634277,-0.451416,-0.268555,-0.085694,0.097168,0.280029,0.462890,0.645752
